# Praktikum 2 - Transformer

Note: the praktikums are for your own practice. They will **not be graded**!

You have around one week to work on it. Then we will go over the solutions together in the praktikum time slots!

Remember to make a copy of this notebook to your own Colab. Changes made directly here will not be stored!

In this exercise, you'll implement a basic encoder-only Transformer architecture with PyTorch. We will start with building the basic building blocks and then integrate them into a fully-fleged Transformer model.


<!-- We train the model to solve a POS-Tagging problem (more on that later). In the previous exercise, you implemented your work in numpy. Now, we will switch to PyTorch, which will track the gradients for us and allows us to focus more on the network itself. -->

**Notice**: Whenenver you see an ellipsis `...`, you're supposed to insert code or text answers.

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

Let's actually start with a few basic functions that we will need throughout the exercise, namely **Softmax** and **ReLu**.

$\text{Softmax}(x_{i}) = \frac{\exp(x_i)}{\sum_j \exp(x_j)}$

$\text{ReLU}(x) = \max(0, x)$

## Transformer Block

A typical transformer block consists of the following
- Multi-Head Attention
- Layer Normalization
- Linear Layer
- Residual Connections

<center><img src="https://i.imgur.com/ZKgcoe4.png" alt="transformer block visualization" width="200">

In the next few subsections, we will build these basic building blocks.

### Multi-Head Attention

Multi-Head Attention concatenates the outputs of several so called **attention heads**.

$\textrm{MHA}(Q,K,V) = \textrm{Concat}(H_1,...,H_h)$

<center><img src="https://www.tensorflow.org/images/tutorials/transformer/multi_head_attention.png" width=300>

One attention head consists of linear projections for each of $Q, K$ and $V$ and an attention mechanism called **Scaled Dot-Product Attention**. The attention mechanism scales down the dot products by $\sqrt{d_k}$.

$\textrm{Attention}(Q,K,V)=\textrm{softmax}(\frac{QK^T}{\sqrt{d_k}})V$



If we assume that $q$ and $v$ are $d_k$-dimensional vectors and its components are independent random variables with mean $0$ and a variance of $d_k$, then their dot product has a mean of $0$ and variance of $d_k$. It is preferred to have a variance of $1$ and that's why we scale them down by $\sqrt{d_k}$.

The dot product $q \cdot v$ resembles a measure of similarity.


<center><img src="https://www.tensorflow.org/images/tutorials/transformer/scaled_attention.png" width="350">

Let's start implementing these components. Note that our classes inherit from PyTorch's `nn.Module`. These modules allow us to hold our parameters and easily move them to the GPU (with `.to(...)`). It also let's us define the computation that is performed at every call, in the `forward()` method. For example, when we have an `Attention` module, initialize it like `attention = Attention(...)`, we are able to call it with `attention(Q, K, V)` (it'll execute the `forward` function in an optimized way).

### Layer Normalization

Layer normalization is when the values are normalized across the feature dimension, independently for each sample in the batch. For that, first calculate mean and standard-deviation across the feature dimension and then scale them appropriately such that the mean is 0 and the standard deviation is 1. Introduce **two sets of learnable parameters**, one for shifting the mean (addition) and one for scaling the variance (multiplication) the normalized features (i.e., two parameters for each feature). Tip: Use `nn.Parameter` for that.

$y_{\textrm{norm}}=\frac{x-\mu}{\sqrt{\sigma+\epsilon}}$

$y=y_{\textrm{norm}}\cdot\beta+\alpha$

<center>
<img src="https://i.stack.imgur.com/E3104.png" alt="visualization of layer norm vs. batch norm" width="420">

### Transformer Block

Here, we bring all ingredients together into a single module. Don't forget to add the residual connections. Let's use a 2-layer MLP with ReLU activation.

In [10]:
def transformer_block(hidden_n: int, heads: int, inputs):
    x = layers.MultiHeadAttention(num_heads=heads, key_dim=hidden_n // heads)(
        inputs, inputs
    )
    x = layers.LayerNormalization()(x + inputs)
    x = layers.Dense(hidden_n, activation="relu")(x)
    x = layers.LayerNormalization()(x + inputs)
    return x


def transformer(emb_n: int, hidden_n: int, n: int = 3, h: int = 2, name: str = "Model"):
    inputs = keras.Input(shape=(None,))
    x = layers.Embedding(emb_n, hidden_n)(inputs)
    for _ in range(n):
        x = transformer_block(hidden_n, h, x)
    outputs = layers.GlobalAveragePooling1D()(x)
    try:
        model = keras.Model(inputs, outputs, name=name)
    except Exception:
        model = keras.Model(inputs, outputs)
    return model


model = transformer(1000, 128, 3, 2, "Transformer")
model.summary()

Model: "Transformer"

KeyboardInterrupt: 

## A Simple Transformer Architecture

Let's stack our transformer blocks and add an embedding layer for a simple transformer architecture. You are allowed to use `nn.Embedding` here.

Part-Of-Speech-Tagging (**POS-Tagging**) is a **sequence labeling problem** where we categorize words in a text in correspondence with a particular part of speech (e.g., "noun" or "adjective"). A few examples and classes are shown in the following table:

|  POS Tag  |  Description  |  Examples  |
|-----------|------------|------------|
|  NN | Noun (singular, common) | mass, wind, ...  |
|  NNP | Noun (singular, proper) | Obama, Liverpool, ...  |
| CD  | Numeral (cardinal)  | 1890, 0.5, ...  |
|  DT | Determiner  | all, any, ... |
| JJ | Adjective (ordinal) | oiled, third, ... |
... many more

### CoNLL2000 Dataset

Let's load our dataset which is the **CoNLL2000 dataset** and look at an example.

In [1]:
from torch.utils.data import Dataset, DataLoader
from torchtext.datasets import CoNLL2000Chunking
import pandas as pd

train_df = pd.DataFrame(CoNLL2000Chunking()[0], columns=['words', 'pos_tags', 'chunk'])
test_df = pd.DataFrame(CoNLL2000Chunking()[1], columns=['words', 'pos_tags', 'chunk'])

train_src, train_tgt = train_df['words'].tolist(), train_df['pos_tags'].tolist()
test_src, test_tgt = test_df['words'].tolist(), test_df['pos_tags'].tolist()

print(train_src[0])
print(train_tgt[0])

c:\Users\matze\anaconda3\envs\cs-tensorflow\Lib\site-packages\torchtext\__init__.py:7: SyntaxWarning: invalid escape sequence '\ '
  "\n/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ \n"


OSError: [WinError 127] Die angegebene Prozedur wurde nicht gefunden

First, we need to create a vocabulary. Our dataset is already tokenized. However, we need to assign ids to them in order to input them to the embedding layer. We also need the number of embeddings (`num_embeddings`) for the size of our lookup table of `nn.Embedding`.

Thus, we will iterate over all sentences replace them with ids and the mapping to our vocabulary. It'll be handy to have two different mappings, from id to token, as well as, from token to id. Note that we will add a special token `<unk>` with id `0` for words that are unknown (that are not in the training dataset but could possibly be in the test dataset).

In [ ]:
vocabulary_id2token : dict = {0: '<unk>'}
vocabulary_token2id : dict = {'<unk>': 0}

for sentence in train_src:
    ...

Let's do the same for our classes:

In [ ]:
classes_id2name : dict = {}
classes_name2id : dict = {}

for sentence in *[train_src, test_src]:
    ...

Now, let's use PyTorch's `Dataset` and `DataLoader` to help us batching our data. Let's also replace tokens and classes with our ids. For that, complete `get_token_ids` and `get_class_ids`.

In [ ]:
def get_token_ids(src: list[str]) -> list[int]:
    ...

def get_class_ids(tgt: list[str]) -> list[int]:
    ...

class ConllDataset(Dataset):
  def __init__(self, src, tgt):
        self.src = src
        self.tgt = tgt

  def __len__(self):
        return len(self.src)

  def __getitem__(self, index):
        src = self.src[index]
        tgt = self.tgt[index]

        return {
            'src': get_token_ids(src),
            'tgt': get_class_ids(tgt),
        }

train_dataset = ConllDataset(train_src, train_tgt)
test_dataset = ConllDataset(test_src, test_tgt)

We will use a **batch size of 32**.

In [ ]:
BATCH_SIZE = 32

However, since our examples are of different length, we need to pad shorter examples to the length of the example with the maximum length in our batch. So, let's define a special **padding token** in our vocabulary:

In [ ]:
padding_token = ...

vocabulary_id2token ...
vocabulary_token2id ...

The `collate_fn` is the function that actually receives a batch and needs to add the padding tokens, then returns `src` and `tgt` as `Tensor`s of size `[B, S]` where `B` is our batch size and `S` our maximum sequence length. This function should additionally return a `mask`, a `Tensor` with binary values to indicate whether the specific element is a padding token or not (0 if it's a padding token, 1 if not), such that we can ignore padding tokens in our attention mechanism and loss calculation.

In [ ]:
def collate_fn(batch: list[dict]) -> dict[str, Tensor]:
    """
    batch: list of dictionaries with keys src and tgt (as defined in ConllDataset)
    """
    ...
    return {
        'src': src,
        'tgt': tgt,
        'mask': mask,
    }

With that, we can use PyTorch's `DataLoader` which will shuffle and batch our data automatically.

In [ ]:
train_data_loader = DataLoader(train_dataset, collate_fn=collate_fn, batch_size=BATCH_SIZE, shuffle=True)
test_data_loader = DataLoader(test_dataset, collate_fn=collate_fn, batch_size=BATCH_SIZE, shuffle=True)

### Architecture

Let's build a transformer model with three layers, three attention heads and an embedding dimension of 128. Also, let's not forget to add a classification head to our model.

In [ ]:
class CoNLL2000Transformer(nn.Module):
    def __init__(self, transformer, ...):
        super().__init__()
        self.transformer = transformer
        self.classification_layer = ...

    def forward(self, X, mask):
        ...

model = CoNLL2000Transformer(Transformer(...), ...)

### Training

Initialize the **AdamW** optimizer from the `torch.optim` module and choose the most appropriate loss function for our task.

In [ ]:
optimizer = ...
criterion = ...

Build a basic training loop and train the network for three epochs.
- Use everything we've built to far, including `train_data_loader`, `model`, `optimizer` and `criterion`.
- At every 50th step print the average loss of the last 50 steps.
- It is suggested to make a basic training procedure to work on the CPU first. Once it successfully runs on the CPU, you can switch to the GPU (click on change runtime and add an hardware accelerator if you use Colab) and run for the whole three epochs. Note: For this to work, you need to transfer the `model` and the input tensors to the GPU memory. This simply works by calling `.to(device)` on the model and tensors, where `device` and either be `cpu` or `cuda` (for the GPU).

In [ ]:
DEVICE = 'cpu' # later replace with 'cuda' for GPU
EPOCHS = 3

model = model.to(device)

for epoch in range(EPOCHS):
    ...

### Evaluation

Let's see what's the accuracy is of our model. Since we already implemented accuracy in the previous exercise, we'll now let you use the torchmetrics package.

In [ ]:
from torchmetrics import Accuracy

accuracy = Accuracy(average='micro')

Calculate the average accuracy of all examples in the test dataset.

In [ ]:
...

Let's also look at the accuracy **for each class separately**:

In [ ]:
...

## Positional Embeddings

The attention mechanism does not consider the position of the tokens which hurts its performance for many problems. We can solve this issue in several ways. We can either add a positional encoding (via trigonometric functions) or we can learn positional embeddings along the way, in a similar way as BERT does. Here, we will add learnable positional embeddings to our exisisting model with another embedding layer.

The longest sequence in our dataset has 78 tokens (you can trust us on that). So, let's set the number of embeddings for our positional embedding layer to that number. Again, you should use `nn.Embedding`.

Copy the inner parts of your `Transformer` class and add positional embeddings to it.

In [ ]:
class TransformerPos(nn.Module):
    def __init__(self, emb_n: int, pos_emb_n: int, hidden_n: int, n:int =3, h:int =2):
        """
        emb_n: number of token embeddings
        pos_emb_n: number of position embeddings
        hidden_n: hidden dimension
        n: number of layers
        h: number of heads per layer
        """
        super().__init__()
        self.positional_embeddings = ...
        ...

    def forward(self):
        ...

In [ ]:
model_pos = CoNLL2000Transformer(TransformerPos(...), ...)

### Training

Same procedure as before. Let's reinitialize our optimizer and our loss function and run the same training loop with our new model `model_pos`.

In [ ]:
optimizer = ....
criterion = ...

In [ ]:
...

### Evaluation

Now, let's check if our performance on the accuracy got improved.

In [ ]:
...

Again, let's also check each class. Which classes got improved the most by adding positional embeddings?

In [ ]:
...

As an optional task, you can play around with the model by switching out the transformer component for other architecture, e.g., LSTM, an observe the change in performance.